# Visualize your Lookout for Metrics on QuickSight
## Introduction
The notebook generates all the necessary resources to bring your inference results and backtest/live data from Amazon Lookout for Metrics (L4M) to Amazon Quickight for visualization. The services used in this notebook are:


-[Amazon QuickSight](https://aws.amazon.com/quicksight/)

-[AWS Glue](https://aws.amazon.com/glue)

-[Amazon Athena](https://aws.amazon.com/athena)

-[Amazon S3](https://aws.amazon.com/s3/)

In [ ]:
import boto3
import time
import uuid 
import json

In [ ]:
# Get AWS Region and AWS Account ID  

USING_SAGEMAKER = True
# Change to region = "us-east-1" for example to use that region if you ARENT using SageMaker
region = None

if USING_SAGEMAKER: 
    with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
        data = json.load(notebook_info)
        resource_arn = data['ResourceArn']
        region = resource_arn.split(':')[3]

region_L4M = region
region_user_Quicksight = region

response = boto3.client('sts').get_caller_identity()
AwsAccountId=response['Account']

In [ ]:
# Define boto3 clients 
clientGlue = boto3.client('glue', region_name=region_L4M)
clientS3 = boto3.client('s3', region_name=region_L4M)
clientQuicksight = boto3.client('quicksight', region_name=region_L4M)

In [ ]:
user_name = 'TeamRole/MasterKey'
response = clientQuicksight.describe_user(
    UserName=user_name,
    AwsAccountId=AwsAccountId,
    Namespace='default'
)
user_ARN = response['User']['Arn']
print(user_ARN)

In [ ]:
# Load parameters
with open('../ml_ops/params.json') as f:
  data = json.load(f)

bucket_name= data['s3_bucket']
# location of data used as input to backtest detector
inference_data_input_path_s3='s3://'+bucket_name+'/ecommerce/backtest/' 
# location of anomalies from backtest 
inference_data_output_Lambda_path_s3='s3://'+bucket_name+'/anomalyResults/'

crawler_role_arn = data['crawler_role_arn']  # /L4M_visualization_glue


In [ ]:
print(crawler_role_arn)

## Amazon Glue
### Database creation
The following API will create a database in Glue in the region you specified previously. The database allows to store the metadata of the tables built by the crawler below. 

In [ ]:
#Database creation
glue_dbname = 'l4m_anomalies_'+str(uuid.uuid4())
response_glueDatabase = clientGlue.create_database(
    DatabaseInput={
        'Name': glue_dbname,
        'Description': 'Database to visualize the L4M results on Quicksight'
    }
)
print(glue_dbname)

### Crawler
Glue Crawler crawls S3 bucket for data files (e.g. CSV files) to create glue tables.
The following API call will create the crawler and the next cell will run the crawler.

In [ ]:
crawler_name = 'L4M_visualize_anomalies' + str(uuid.uuid4())
response_crawler = clientGlue.create_crawler(
    Name=crawler_name,
    Role= crawler_role_arn,
    DatabaseName=glue_dbname,
    Description= 'Crawler - scan anomalies to visualize L4M results',
    Targets={
        'S3Targets':[
            {
                'Path': inference_data_input_path_s3
            },
            {
                'Path': inference_data_output_Lambda_path_s3
            }
        ]
    }
)
print(crawler_name)

In [ ]:
clientGlue.start_crawler(Name=crawler_name)

In [ ]:
# check crawler job status 
finish=False
while finish==False:
    
    response = clientGlue.get_crawler(
        Name=crawler_name,        
    )
    if response['Crawler']['State']=='STOPPING':
        time.sleep(20)
        print('Crawler has finished')
        finish=True
    else:
        print('Crawler is running')
        time.sleep(10)

In [ ]:
# Populate data from Glue tables. this will be used for setting up Quicksight datasets
response_glue_tables = clientGlue.get_tables(
    DatabaseName=glue_dbname
)
s= response_glue_tables['TableList']
table_dimensioncontributions = next((item for item in s if item['Name'] == 'dimensioncontributions'), None)
table_metricvalueAnomalyscore = next((item for item in s if item['Name'] == 'metricvalue_anomalyscore'), None)
table_liveData = next((item for item in s if item['Name'] == 'backtest'), None)

#Formating the data to fit the dataset for QuickSight
for item in table_dimensioncontributions['StorageDescriptor']['Columns']:
    item['Type'] =  item['Type'].upper()
    if item['Type']=='DOUBLE' or item['Type']=='BIGINT':
        item['Type']='DECIMAL'
        
for item in table_metricvalueAnomalyscore['StorageDescriptor']['Columns']:
    item['Type'] =  item['Type'].upper()
    if item['Type']=='DOUBLE' or item['Type']=='BIGINT':
        item['Type']='DECIMAL'
        
for item in table_liveData['StorageDescriptor']['Columns']:
    item['Type'] =  item['Type'].upper()
    if item['Type']=='DOUBLE' or item['Type']=='BIGINT':
        item['Type']='DECIMAL'

## Amazon QuickSight - Visualization in 3 steps 
1/ Define data source
2/ Create Dataset
3/ Create and visualize analysis

In [ ]:
dsId = 'L4M_ds_' + str(uuid.uuid4())
response_datasource = clientQuicksight.create_data_source(
    AwsAccountId=AwsAccountId,
    DataSourceId=dsId,
    Name='L4M_Visualization',
    Type='ATHENA',
    Permissions=[
        {
            'Principal': user_ARN,
            'Actions': [               
                'quicksight:DescribeDataSource',
                'quicksight:DescribeDataSourcePermissions',
                'quicksight:UpdateDataSource',
                'quicksight:UpdateDataSourcePermissions',
                'quicksight:DeleteDataSource',
                'quicksight:PassDataSource',
            ]
        },
    ],
    
)
print('Data source ID :', dsId)

### Dataset creation
We will generate a dataset by using the API call [create_data_set()](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/quicksight.html#QuickSight.Client.create_data_set) below. We join the tables *backtestData* and *metricvalueAnomalyscore*. The variable *colummns_to_be_shown* allows to choose what metrics you would like to see in your analysis.

In [ ]:
colummns_to_be_shown = ['timestamp','marketplace','platform','views','revenue','revenueanomalymetricvalue',
                        'revenuegroupscore','viewsanomalymetricvalue','viewsgroupscore']


response_dataset_live = clientQuicksight.create_data_set(
    AwsAccountId= AwsAccountId,
    DataSetId='L4M_liveData'+str(uuid.uuid4()),
    Name='L4M_Visualization_dataset_with_backtestData',
    PhysicalTableMap={
        'liveData': {
            'RelationalTable': {
                'DataSourceArn': response_datasource['Arn'],
                'Catalog': 'AwsDataCatalog',
                'Schema': glue_dbname,
                'Name': 'backtest',
                'InputColumns': table_liveData['StorageDescriptor']['Columns']                
            }          
        },
        'metricvalueAnomalyscore': {
            'RelationalTable': {
                'DataSourceArn': response_datasource['Arn'],
                'Catalog': 'AwsDataCatalog',
                'Schema': glue_dbname,
                'Name': 'metricvalue_anomalyscore',
                'InputColumns': table_metricvalueAnomalyscore['StorageDescriptor']['Columns']                
            }          
        },
    },
    
    LogicalTableMap={
        'liveData': {
            'Alias': 'alias liveData',
            'Source':{
                'PhysicalTableId': 'liveData'
            },
            'DataTransforms': [
                {
                    'RenameColumnOperation': {
                        'ColumnName': 'timestamp',
                        'NewColumnName': 'timestampInput'
                    }
                }
            ],
            
        },
        'metricvalueAnomalyscore':{
            'Alias': 'alias metricvalueAnomalyscore',
            'Source':{
                'PhysicalTableId': 'metricvalueAnomalyscore'
            },
            'DataTransforms': [
                {
                    'RenameColumnOperation': {
                        'ColumnName': 'timestamp',
                        'NewColumnName': 'timestampAnomalyscore'
                    }
                },
                {
                    'RenameColumnOperation': {
                        'ColumnName': 'marketplace',
                        'NewColumnName': 'marketplace_anomalyScore'
                    }
                },
                {
                    'RenameColumnOperation': {
                        'ColumnName': 'platform',
                        'NewColumnName': 'platform_anomalyScore'
                    }
                }
            ],
        },
      
        'joinWithRight':{
            'Alias': 'RIGHT join of input and output',
            'Source': {
                'JoinInstruction': {
                    'LeftOperand': 'liveData',
                    'RightOperand': 'metricvalueAnomalyscore',
                    'Type': 'LEFT',
                    'OnClause': ' timestampInput = timestampAnomalyscore AND marketplace = marketplace_anomalyScore AND platform= platform_anomalyScore' 
                },
                
            },
            'DataTransforms': [
                {
                    'CreateColumnsOperation': {
                        'Columns': [
                            {
                                'ColumnName': 'timestamp',
                                'ColumnId': 'calculatedTimestamp',
                                'Expression': 'parseDate(timestampInput,"yyyy-MM-dd HH:mm:ss")'
                            },
                        ]
                        
                    },
                },
                
                    {
                        'ProjectOperation': {
                            'ProjectedColumns': colummns_to_be_shown
                    }
                    }
     
                
    
            ],
        },
        
        
    },
    ImportMode='DIRECT_QUERY',    
    Permissions= [
      {
        'Principal': user_ARN,
        "Actions": [
                    "quicksight:DescribeDataSet",
                    "quicksight:DescribeDataSetPermissions",
                    "quicksight:PassDataSet",
                    "quicksight:DescribeIngestion",
                    "quicksight:ListIngestions",
                    'quicksight:UpdateDataSet',
                    "quicksight:DeleteDataSet"
            ]
        
      } 
    ],
)



The next step is to build the dashboards of the data you would like to see)